In [1]:
#Build a simple LLM Application with LCEL

# In this quickstart we'll show you how to build a simple
# LLM application with langchain. This application translate text from English into another language.
# This is a relatively simple LLM application - it's just a single 
# LLM call plus some prompting. Still, this a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM Call!


# Topics
# LLM model, Prompt template &  Outputparser, LCEL to chain components together, Tracing app with langsmith, deploying with langserve

In [2]:
## Open AI API and Open Source Models -- Llama3, Gemma2, Mistral -- Groq API Key
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [5]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [7]:
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020F47E54AC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020F47E556C0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [9]:

# it is used to mention which msg is provided by human and which by system
from langchain_core.messages import HumanMessage, SystemMessage

In [14]:
## Method-1: Hard coding values to get output (Not a generic way)
messages = [
    SystemMessage(content="Translate the following from Englist to Hindi"),
    HumanMessage(content="Hello how are you?")
]

result = model.invoke(messages)
result

AIMessage(content='The correct translation is: \n\n**Bonjour, comment allez-vous ?** \n\nHere\'s why:\n\n* **Bonjour** means "Hello"\n* **Comment allez-vous?** means "How are you?" (formal)\n\nLet me know if you\'d like to know other ways to say "How are you?" in French!\n\n\n', response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 22, 'total_tokens': 98, 'completion_time': 0.138181818, 'prompt_time': 0.001875973, 'queue_time': 0.578498383, 'total_time': 0.140057791}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb69a7e2-92c2-4db3-a5f1-876c3c248c1f-0', usage_metadata={'input_tokens': 22, 'output_tokens': 76, 'total_tokens': 98})

In [16]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'The correct translation is: \n\n**Bonjour, comment allez-vous ?** \n\nHere\'s why:\n\n* **Bonjour** means "Hello"\n* **Comment allez-vous?** means "How are you?" (formal)\n\nLet me know if you\'d like to know other ways to say "How are you?" in French!\n\n\n'

In [17]:
### Using LCEL - Chain the components
chain = model|parser
chain.invoke(messages)

'The correct French translation of "Hello, how are you?" is:\n\n**Bonjour, comment allez-vous ?** \n\nLet me know if you\'d like to translate any other phrases! \n'

In [19]:
## Method-2: Prompt Template (Instead of passing list of messages use prompt template)

from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following into {language}: "
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [21]:
result = prompt.invoke({"language": "French", "text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French: '), HumanMessage(content='Hello')])

In [22]:
result.to_messages()

[SystemMessage(content='Translate the following into French: '),
 HumanMessage(content='Hello')]

In [23]:
# Chaining together components with LCEL

chain = prompt|model|parser
chain.invoke({"language":"French", "text":"Hello"})

'Bonjour \n'